In [29]:
import pandas as pd
import numpy as np
import urllib.request, json 
from urllib.parse import quote

In [30]:
df = pd.read_excel('data/top100_richest_people_forbes_ua.xlsx', sheet_name='Sheet1')
df.head(5)

,Місце,"Ім'я, прізвище",Статки,Компанія/бренд,Сфера інтересів
0,1,Рінат Ахметов,"$7,6 млрд","«Метінвест», ДТЕК","Металургія, ПЕК"
1,2,Віктор Пінчук,"$2,5 млрд",«Інтерпайп»,"Металургія, нерухомість"
2,3,Костянтин Жеваго,"$2,4 млрд",Ferrexpo,Металургія
3,4,Ігор Коломойський,"$1,8 млрд",«Укрнафта»,"ЕК, інвестиції"
4,5,Геннадій Боголюбов,"$1,7 млрд",«Укрнафта»,"ПЕК, інвестиції"


In [31]:
#defining columns in English
col_names = ['Rank', 'Full name', 'Capital', 'Company/Brand', 'Branch']
#renaming columns
df = pd.DataFrame(data=df.values, columns=col_names)

#translating Branch values with Google Translate api
translation = []
for i in df['Branch']:
    with urllib.request.urlopen("https://translate.googleapis.com/translate_a/single?client=gtx&sl=uk&tl=en&dt=t&q="+quote(i)) as url:
        transl = json.load(url)[0][0][0]
    translation.append(transl)

#replacing in the dataset
df['Branch'] = translation
df.head()


,Rank,Full name,Capital,Company/Brand,Branch
0,1,Рінат Ахметов,"$7,6 млрд","«Метінвест», ДТЕК","Metallurgy, PEK"
1,2,Віктор Пінчук,"$2,5 млрд",«Інтерпайп»,"Metallurgy, real estate"
2,3,Костянтин Жеваго,"$2,4 млрд",Ferrexpo,Metallurgy
3,4,Ігор Коломойський,"$1,8 млрд",«Укрнафта»,"EC, investments"
4,5,Геннадій Боголюбов,"$1,7 млрд",«Укрнафта»,"PEK, investments"
